# CNN + Data augmentation

## Equipo 4:


*   Karla Andrea Palma Villanueva (A01754270)
*   Viviana Alanis Fraige (A01236316)
* David Fernando Armendariz Torres (A01570813)
* Alan Alberto Mota Yescas (A01753924)
* Adrián Chávez Morales (A01568679)
* Jose Manuel Armendáriz Mena (A01197583)

### Introducción

El objetivo de este notebook es implementar un modelo de Red Neuronal Convolucional (CNN) utilizando el dataset CIFAR-10 para la tarea de clasificación de imágenes. CIFAR-10 es un conjunto de datos ampliamente utilizado en el campo del aprendizaje profundo, que consta de 60,000 imágenes a color distribuidas en 10 categorías, tales como aviones, automóviles, pájaros y gatos. Cada imagen tiene un tamaño de 32x32 píxeles y está etiquetada con su respectiva clase.

Además de desarrollar el modelo CNN, aplicaremos técnicas de Data Augmentation (aumento de datos) para incrementar la variedad del conjunto de entrenamiento. Esto nos permitirá reducir el riesgo de sobreajuste (overfitting) y mejorar la capacidad de generalización del modelo en datos no vistos.

Al finalizar este trabajo, analizaremos el rendimiento del modelo evaluando métricas clave como la precisión (accuracy) y la pérdida (loss), comparando los resultados con y sin Data Augmentation. Nuestro objetivo final es demostrar que el uso de estas técnicas contribuye a un modelo más robusto y eficiente.

### Exploración, explicación y limpieza de datos

#### Origen y Contexto del Dataset CIFAR-10

El dataset CIFAR-10 fue desarrollado por Alex Krizhevsky, Geoffrey Hinton y Vinod Nair como parte del trabajo de investigación del grupo de Aprendizaje Profundo en la Universidad de Toronto. Es uno de los datasets más utilizados para experimentos en el campo de la visión por computadora debido a su simplicidad, pero al mismo tiempo, a su capacidad de ofrecer desafíos relevantes para la construcción de modelos robustos.

Este conjunto de datos contiene 60,000 imágenes en color, de 32x32 píxeles cada una, distribuidas en 10 categorías diferentes:
- Aviones
- Automóviles
- Pájaros
- Gatos
- Ciervos
- Perros
- Ranas
- Caballos
- Barcos
- Camiones

Cada categoría tiene 6,000 imágenes, divididas en 50,000 imágenes para entrenamiento y 10,000 imágenes para pruebas. Esta organización permite desarrollar y evaluar modelos de clasificación de imágenes con un tamaño de dataset razonable para tareas académicas o de investigación.

El objetivo del CIFAR-10 es facilitar la experimentación en tareas de reconocimiento de patrones y clasificación de imágenes en situaciones donde las imágenes contienen objetos simples, pero con variaciones en poses, colores, y ángulos. Además, al ser un dataset equilibrado (con la misma cantidad de imágenes por clase), permite evaluar de manera precisa el desempeño de los modelos sin sesgos hacia una clase específica.

La elección de este dataset es especialmente relevante en el campo del aprendizaje profundo, ya que se utiliza frecuentemente para enseñar conceptos fundamentales como Redes Neuronales Convolucionales (CNN) y técnicas de aumento de datos (Data Augmentation), las cuales son fundamentales para mejorar la generalización del modelo y prevenir el sobreajuste. 

CIFAR-10 es de acceso libre y está disponible en varios recursos de aprendizaje de TensorFlow y PyTorch, lo que facilita su uso para fines educativos y de investigación.


In [3]:
#Importación de librerías
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# Fijar la semilla 
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [5]:
# Cargar el dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalización del dataset 
mean = np.mean(x_train, axis=(0, 1, 2, 3))
std = np.std(x_train, axis=(0, 1, 2, 3))
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

print(f"Tamaño del conjunto de entrenamiento: {x_train.shape[0]}")
print(f"Tamaño del conjunto de prueba: {x_test.shape[0]}")
print(f"Forma de las imágenes: {x_train.shape[1:]}")
print(f"Cantidad de clases: {len(np.unique(y_train))}")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step
Tamaño del conjunto de entrenamiento: 50000
Tamaño del conjunto de prueba: 10000
Forma de las imágenes: (32, 32, 3)
Cantidad de clases: 10


Al cargar el dataset CIFAR-10, observamos las siguientes características:

Conjunto de Entrenamiento:

Tamaño: 50,000 imágenes
Cada imagen tiene una forma de 32x32 píxeles y 3 canales de color (RGB), lo que implica que cada imagen es una matriz tridimensional de tamaño (32, 32, 3).
Conjunto de Prueba:

Tamaño: 10,000 imágenes
Las imágenes tienen la misma estructura que las del conjunto de entrenamiento, es decir, (32, 32, 3).
Etiquetas de Clase:
El dataset contiene 10 categorías de imágenes diferentes. Las etiquetas se encuentran en los arrays y_train y y_test para los conjuntos de entrenamiento y prueba, respectivamente. Cada etiqueta es un número entero que representa una categoría específica.

El dataset CIFAR-10 no requiere procesos de limpieza adicionales, ya que se trata de un conjunto de datos ampliamente utilizado y cuidadosamente curado para tareas de aprendizaje profundo. Los datos se presentan de forma consistente, sin valores faltantes, duplicados o registros incompletos. Además, cada imagen está correctamente etiquetada y alineada con su categoría correspondiente. Esto garantiza que todas las observaciones estén listas para ser utilizadas en el modelo sin necesidad de aplicar procesos de limpieza convencionales, como la eliminación de valores nulos o el tratamiento de datos inconsistentes.

Sin embargo, aunque la limpieza no es necesaria, sí realizamos transformaciones fundamentales para optimizar el rendimiento del modelo. La primera transformación aplicada fue la normalización de las imágenes, que consiste en restar la media y dividir por la desviación estándar de los valores de los píxeles en el conjunto de entrenamiento. Esto es crucial para asegurar que las intensidades de los píxeles, originalmente en un rango de 0 a 255, estén distribuidas en torno a una media de 0 con una desviación estándar de 1. La normalización mejora la eficiencia del entrenamiento al acelerar la convergencia del modelo y prevenir problemas de escala que podrían afectar la optimización de los parámetros.

Otra transformación importante que implementaremos es el aumento de datos (Data Augmentation), una técnica que incrementa artificialmente la variedad del conjunto de entrenamiento. Esta técnica genera nuevas muestras a partir de las imágenes existentes mediante modificaciones como rotaciones aleatorias, traslaciones, inversiones horizontales, cambios en el brillo y la introducción de ruido. El Data Augmentation es esencial para evitar el sobreajuste del modelo, ya que ayuda a mejorar su capacidad de generalización al exponerlo a una mayor diversidad de patrones durante el entrenamiento. Esto permite que el modelo aprenda de manera más robusta y tenga un mejor desempeño al enfrentar nuevas imágenes no vistas. 

Estas transformaciones aseguran que los datos estén en condiciones óptimas para ser procesados por la Red Neuronal Convolucional, lo que facilita un entrenamiento eficiente y contribuye a construir un modelo con mayor capacidad de generalización.

### Desarrollo del Modelo de Deep Learning

La implementación del modelo CNN en este proyecto sigue una arquitectura cuidadosamente diseñada para mejorar el rendimiento en la clasificación del dataset CIFAR-10. El modelo se compone de múltiples capas convolucionales organizadas en tres bloques, cada uno con dos capas convolucionales, seguidas de normalización por lotes (*Batch Normalization*), capas de agrupamiento máximo (*MaxPooling*) y capas de abandono (*Dropout*) para mitigar el sobreajuste. Estas decisiones permiten capturar características complejas de las imágenes y asegurar que el modelo pueda generalizar adecuadamente en datos no vistos. La adición de más capas en comparación con arquitecturas básicas de CNN contribuye a la creación de un modelo más profundo y robusto, capaz de aprender representaciones más complejas.

La arquitectura del modelo se resume en tres bloques convolucionales progresivos que aumentan la profundidad del modelo, comenzando con 64 filtros en el primer bloque, 128 en el segundo y 256 en el tercero. Cada bloque termina con una capa de agrupamiento (*MaxPooling*) para reducir las dimensiones espaciales y una capa de abandono (*Dropout*) que ayuda a evitar el sobreajuste mediante la desactivación aleatoria de neuronas durante el entrenamiento. Tras los bloques convolucionales, se utiliza una capa de *Global Average Pooling* que reduce las dimensiones del espacio de características sin perder información relevante, seguida de una capa densa de 256 neuronas. Finalmente, la capa de salida tiene 10 neuronas con activación *softmax*, lo que permite que el modelo prediga la probabilidad de pertenencia a cada una de las 10 clases del dataset CIFAR-10.

In [6]:
# Definición del modelo CNN 
def create_deeper_cnn_model():
    model = Sequential()

    # Primera capa convolucional
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.2))

    # Segunda capa convolucional
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.3))

    # Tercera capa convolucional
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Dropout(0.4))

    model.add(GlobalAveragePooling2D())

    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    # Capa de salida
    model.add(Dense(10, activation='softmax'))

    return model

# modelo
model = create_deeper_cnn_model()
model.summary()

c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,217,354 (4.64 MB)

 Trainable params: 1,215,562 (4.64 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [7]:
optimizer = RMSprop(learning_rate=0.0005)
# modelo usando SparseCategoricalCrossentropy
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [9]:
# Ajustar el generador al conjunto de entrenamiento
datagen.fit(x_train)

# Definir los Callbacks
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True
)

def scheduler(epoch, lr):
    if epoch < 20:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_callback = LearningRateScheduler(scheduler)

La implementación del modelo CNN en este proyecto no solo se basa en una arquitectura profunda y bien estructurada, sino que también incluye diversas estrategias para optimizar el entrenamiento y mejorar el rendimiento general. La configuración del optimizador y la función de pérdida son fundamentales para garantizar una convergencia eficiente del modelo. Se utilizó el optimizador RMSprop con una tasa de aprendizaje de 0.0005, lo que ayuda a ajustar los pesos del modelo de manera efectiva en cada iteración. La función de pérdida seleccionada es Sparse Categorical Crossentropy, adecuada para tareas de clasificación con etiquetas codificadas como enteros. Además, se utiliza la métrica de precisión (*accuracy*) para evaluar el desempeño del modelo durante el entrenamiento y la validación.

Un elemento clave en la implementación es la inclusión del Data Augmentation mediante la clase ImageDataGenerator, que genera variaciones aleatorias de las imágenes de entrenamiento. Esto incluye rotaciones, traslaciones horizontales y verticales, zoom, cizallamiento y volteo horizontal. Estas técnicas amplían artificialmente el conjunto de datos, ayudando a que el modelo aprenda patrones más diversos y evitando el sobreajuste. La integración del generador con el conjunto de entrenamiento se realiza mediante el método fit(), lo que permite que las transformaciones sean aplicadas en tiempo real durante el entrenamiento del modelo.

Para mejorar aún más la eficiencia del proceso de entrenamiento, se han configurado callbacks que controlan la dinámica del entrenamiento. Entre estos se encuentra ReduceLROnPlateau, que reduce la tasa de aprendizaje si no se observa mejora en la pérdida de validación durante un número determinado de épocas, permitiendo que el modelo afine mejor los parámetros en las últimas etapas del entrenamiento. También se ha implementado EarlyStopping para detener el entrenamiento de manera anticipada si la precisión de validación deja de mejorar, evitando el uso innecesario de recursos computacionales y sobreentrenamiento. Finalmente, se añade un programador de la tasa de aprendizaje mediante LearningRateScheduler, que ajusta la tasa de aprendizaje en función del número de épocas, acelerando la convergencia en las primeras etapas y refinando el aprendizaje en las etapas finales. 

Estos componentes trabajan en conjunto para construir un modelo CNN eficiente, robusto y capaz de generalizar adecuadamente en tareas de clasificación de imágenes.

In [10]:
# Entrenar el modelo
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    validation_data=(x_test, y_test),
    steps_per_epoch=x_train.shape[0] // 64,
    epochs=20,
    callbacks=[reduce_lr, early_stopping, lr_callback],
    verbose=1
)

c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 326s 410ms/step - accuracy: 0.3344 - loss: 1.8741 - val_accuracy: 0.5381 - val_loss: 1.3789 - learning_rate: 5.0000e-04
Epoch 2/20
  1/781 ━━━━━━━━━━━━━━━━━━━━ 4:47 369ms/step - accuracy: 0.5000 - loss: 1.3376

c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


781/781 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.5000 - loss: 1.3376 - val_accuracy: 0.5203 - val_loss: 1.4593 - learning_rate: 5.0000e-04
Epoch 3/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 277s 355ms/step - accuracy: 0.5469 - loss: 1.2790 - val_accuracy: 0.6085 - val_loss: 1.1443 - learning_rate: 5.0000e-04
Epoch 4/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.5469 - loss: 1.0697 - val_accuracy: 0.5929 - val_loss: 1.2174 - learning_rate: 5.0000e-04
Epoch 5/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 377s 483ms/step - accuracy: 0.6172 - loss: 1.0973 - val_accuracy: 0.6828 - val_loss: 0.9173 - learning_rate: 5.0000e-04
Epoch 6/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.5938 - loss: 1.0473 - val_accuracy: 0.6839 - val_loss: 0.9157 - learning_rate: 5.0000e-04
Epoch 7/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 471s 603ms/step - accuracy: 0.6653 - loss: 0.9697 - val_accuracy: 0.6860 - val_loss: 0.9520 - learning_rate: 5.0000e-04
Epoch 8/20
781/781 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - a

Los resultados obtenidos durante el entrenamiento del modelo muestran un progreso constante en la precisión tanto del conjunto de entrenamiento como del conjunto de validación a lo largo de las 20 épocas. Al inicio del entrenamiento, la precisión en el conjunto de entrenamiento es baja (50%) y la pérdida es alta (1.3376), lo que es esperado al tratarse de las primeras iteraciones del modelo en la tarea de clasificación. A medida que avanzan las épocas, el modelo mejora progresivamente en ambas métricas. Por ejemplo, hacia la época 10, la precisión del conjunto de validación alcanza 74.5% y la pérdida disminuye a 0.7378, lo que indica que el modelo está aprendiendo patrones significativos de los datos.

Un aspecto notable del entrenamiento es la activación del callback ReduceLROnPlateau en la época 14. Este mecanismo reduce la tasa de aprendizaje a la mitad, lo que permite al modelo ajustar los pesos de manera más fina en las últimas etapas del entrenamiento, evitando grandes fluctuaciones en la pérdida. Después de este ajuste, la precisión del conjunto de validación sigue mejorando, alcanzando 82.15% en la última época con una pérdida de 0.5332. Esto sugiere que el modelo fue capaz de generalizar bien sin caer en problemas de sobreajuste, dado que las diferencias entre la precisión del entrenamiento y la validación son relativamente pequeñas.



In [11]:
# Evaluación del modelo
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc:.4f}')

313/313 - 15s - 48ms/step - accuracy: 0.8215 - loss: 0.5332

Test accuracy: 0.8215


In [12]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Graficar la historia del entrenamiento
def plot_history(history):
    fig = make_subplots(rows=1, cols=2, 
                        subplot_titles=('Pérdida durante el entrenamiento', 
                                        'Precisión durante el entrenamiento'))

    fig.add_trace(go.Scatter(
        x=list(range(len(history.history['loss']))),
        y=history.history['loss'],
        mode='lines+markers',
        name='Loss'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=list(range(len(history.history['val_loss']))),
        y=history.history['val_loss'],
        mode='lines+markers',
        name='Val Loss'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=list(range(len(history.history['accuracy']))),
        y=history.history['accuracy'],
        mode='lines+markers',
        name='Accuracy'
    ), row=1, col=2)

    fig.add_trace(go.Scatter(
        x=list(range(len(history.history['val_accuracy']))),
        y=history.history['val_accuracy'],
        mode='lines+markers',
        name='Val Accuracy'
    ), row=1, col=2)

    fig.update_layout(
        title_text='Historia del Entrenamiento',
        height=400, width=800
    )

    fig.show()

plot_history(history)


### Resultados e Interpretación 1 

Las gráficas presentadas ilustran la evolución de las métricas de pérdida y precisión a lo largo del entrenamiento del modelo CNN, tanto para el conjunto de entrenamiento como para el de validación. Estas visualizaciones son fundamentales para evaluar el rendimiento del modelo y verificar si está aprendiendo de manera efectiva sin incurrir en problemas de sobreajuste.

En la gráfica de pérdida durante el entrenamiento (a la izquierda), observamos cómo la pérdida disminuye progresivamente en ambas curvas, tanto para el conjunto de entrenamiento como para el conjunto de validación. Esto indica que el modelo está aprendiendo patrones significativos de los datos y mejorando en la minimización de los errores de predicción. Sin embargo, al final del proceso (épocas 19-20), la curva de pérdida para el conjunto de validación muestra un ligero aumento, lo que podría ser un indicio de que el modelo comienza a sobreajustarse. Esta situación es importante monitorearla para evitar que el modelo pierda capacidad de generalización.

Por otro lado, la gráfica de precisión durante el entrenamiento (a la derecha) muestra un incremento constante en la precisión tanto del conjunto de entrenamiento como del conjunto de validación. Desde un valor inicial cercano al 50%, la precisión de validación alcanza aproximadamente 82% en la última época, lo que indica un desempeño sólido. A lo largo de las primeras 10 épocas, se observa un crecimiento acelerado en la precisión, lo que sugiere que el modelo ha encontrado rápidamente patrones útiles. Después de la época 15, el crecimiento se estabiliza, mostrando que el modelo está refinando los últimos ajustes.

En general, ambas gráficas muestran una convergencia adecuada, con mejoras continuas en las métricas de entrenamiento y validación. El uso de callbacks, como ReduceLROnPlateau y EarlyStopping, ha permitido que el modelo ajuste sus parámetros de manera más eficiente. Aunque las curvas de validación no presentan grandes fluctuaciones, el ligero aumento en la pérdida al final del entrenamiento sugiere que podría explorarse una reducción adicional en la tasa de aprendizaje o el uso de más datos para evitar sobreajuste.

### Ajuste de hiperparámetros

La aplicación de ajustes de hiperparámetros en un modelo CNN es fundamental para mejorar su rendimiento y optimizar su capacidad de generalización. En este proyecto, se ha implementado un proceso de **sintonización automática de hiperparámetros utilizando keras_tuner, lo que permite explorar múltiples configuraciones del modelo y seleccionar aquellas que proporcionan los mejores resultados en términos de precisión en el conjunto de validación. La importancia de estos ajustes radica en que los hiperparámetros, como el número de filtros en las capas convolucionales, el tamaño de las capas densas, las tasas de aprendizaje, y el dropout, afectan directamente la capacidad del modelo para aprender patrones significativos sin sobreajustarse a los datos de entrenamiento.

En este modelo, se han utilizado varias configuraciones dinámicas, como la búsqueda del número óptimo de filtros en las capas convolucionales (variando entre 32 y 256) y el ajuste de las unidades de la capa densa final. Además, la inclusión de Data Augmentation con variaciones en rotación, desplazamiento, zoom y brillo contribuye a enriquecer el conjunto de datos y a mejorar la capacidad de generalización del modelo. El uso de learning rate adaptable permite que el optimizador Adam ajuste los pesos de manera más precisa durante el proceso de entrenamiento, evitando que el modelo converja demasiado rápido hacia un mínimo local.

Otro ajuste importante es la integración de un modelo preentrenado, ResNet50, para aprovechar los pesos de una red profunda entrenada en el dataset ImageNet. Esta técnica de transfer learning permite reutilizar características previamente aprendidas en un nuevo contexto, lo que acelera el proceso de entrenamiento y mejora la precisión. Al congelar las capas de la red base, el modelo se enfoca en aprender patrones específicos del dataset CIFAR-10 sin modificar las características generales ya adquiridas. 


In [12]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2]
)

datagen.fit(x_train)


def build_model(hp):
    model = Sequential()

    hp_filters_1 = hp.Int('filters_1', min_value=32, max_value=128, step=32)
    model.add(Conv2D(hp_filters_1, (3, 3), activation='relu', input_shape=(32, 32, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), padding='same'))

    hp_filters_2 = hp.Int('filters_2', min_value=64, max_value=256, step=64)
    model.add(Conv2D(hp_filters_2, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2), padding='same'))

    model.add(GlobalAveragePooling2D())

    hp_units = hp.Int('units', min_value=64, max_value=256, step=64)
    model.add(Dense(hp_units, activation='relu'))

    hp_dropout = hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)
    model.add(Dropout(hp_dropout))

    model.add(Dense(10, activation='softmax'))

    hp_learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,  
    executions_per_trial=1,  
    directory='my_dir',
    project_name='cifar10_tuning'
)

tuner.search(datagen.flow(x_train, y_train, batch_size=64),
             validation_data=(x_test, y_test),
             steps_per_epoch=x_train.shape[0] // 64,
             epochs=10)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Mejores hiperparámetros: {best_hps.values}")

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', patience=10, restore_best_weights=True
)



def build_model(hp):
    model = Sequential()

    hp_filters_1 = hp.Int('filters_1', min_value=64, max_value=128, step=32)
    model.add(Conv2D(hp_filters_1, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    hp_filters_2 = hp.Int('filters_2', min_value=64, max_value=256, step=64)
    model.add(Conv2D(hp_filters_2, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    model.add(GlobalAveragePooling2D())
    hp_units = hp.Int('units', min_value=128, max_value=256, step=64)
    model.add(Dense(hp_units, activation='relu'))
    model.add(Dropout(0.3)) 

    model.add(Dense(10, activation='softmax'))

    hp_learning_rate = hp.Float('learning_rate', min_value=1e-3, max_value=1e-2, sampling='log')
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model







Reloading Tuner from my_dir\cifar10_tuning\tuner0.json
Mejores hiperparámetros: {'filters_1': 96, 'filters_2': 192, 'units': 192, 'dropout': 0.2, 'learning_rate': 0.002312113349326375}


In [13]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False  

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


In [14]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    batch_size=64,
    epochs=10
)


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 434s 470ms/step - accuracy: 0.1116 - loss: 2.3674 - val_accuracy: 0.1717 - val_loss: 2.1957
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 413s 528ms/step - accuracy: 0.1397 - loss: 2.2196 - val_accuracy: 0.2173 - val_loss: 2.1469
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 418s 535ms/step - accuracy: 0.1521 - loss: 2.1940 - val_accuracy: 0.2293 - val_loss: 2.1246
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 414s 529ms/step - accuracy: 0.1524 - loss: 2.1863 - val_accuracy: 0.2490 - val_loss: 2.0795
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 397s 507ms/step - accuracy: 0.1573 - loss: 2.1748 - val_accuracy: 0.2344 - val_loss: 2.0862
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 429s 549ms/step - accuracy: 0.1619 - loss: 2.1611 - val_accuracy: 0.2463 - val_loss: 2.0455
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 412s 526ms/step - accuracy: 0.1629 - loss: 2.1591 - val_accuracy: 0.2510 - val_loss: 2.0189
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 389s 497ms/step - accuracy: 0.1677 -

Los resultados del entrenamiento mostrados reflejan que la configuración del modelo no fue la más adecuada, lo que afectó significativamente el desempeño y la eficiencia del proceso. A pesar de completar las 10 épocas de entrenamiento, los valores de precisión tanto en el conjunto de entrenamiento como en el de validación se mantuvieron extremadamente bajos, con una precisión final de 16.65% en el entrenamiento y 24.71% en la validación. Estos resultados indican que el modelo no fue capaz de aprender correctamente los patrones presentes en los datos, sugiriendo que la arquitectura y los hiperparámetros seleccionados no lograron capturar la complejidad del problema.

Además, la pérdida elevada en ambas curvas confirma que el modelo no logró optimizar de manera efectiva las predicciones, lo que podría indicar un mal ajuste en los hiperparámetros, como el número de filtros, unidades densas, o la tasa de aprendizaje. También es posible que la arquitectura del modelo no sea lo suficientemente profunda o compleja para este tipo de tarea, o que los datos no se hayan preprocesado y utilizado de forma adecuada. 

Otro problema destacado es el tiempo excesivo de entrenamiento, con un promedio de más de 4 minutos por época, lo que resulta ineficiente para obtener resultados subóptimos. Este tiempo prolongado sugiere que el modelo podría estar sobredimensionado en relación al problema o que se están utilizando configuraciones innecesariamente costosas en términos de recursos computacionales, como un tamaño de lote o arquitectura no ajustados.

Dado el rendimiento limitado y la larga duración del entrenamiento, se recomienda una revisión completa de la configuración del modelo. Esto incluiría:
- Ajustar el tamaño de las capas convolucionales y densas.
- Experimentar con tasas de aprendizaje más bajas o utilizando estrategias adaptativas como ReduceLROnPlateau.
- Aumentar las épocas solo si se obtiene una mejor convergencia con los ajustes realizados.
- Considerar arquitecturas preentrenadas (como ResNet o MobileNet) para mejorar el rendimiento mediante transfer learning.



In [15]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("Pérdida (Loss)", "Precisión (Accuracy)"))

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(train_loss) + 1)

fig.add_trace(go.Scatter(x=list(epochs), y=train_loss, mode='lines+markers', name='Train Loss'), row=1, col=1)
fig.add_trace(go.Scatter(x=list(epochs), y=val_loss, mode='lines+markers', name='Val Loss'), row=1, col=1)

fig.add_trace(go.Scatter(x=list(epochs), y=train_acc, mode='lines+markers', name='Train Accuracy'), row=1, col=2)
fig.add_trace(go.Scatter(x=list(epochs), y=val_acc, mode='lines+markers', name='Val Accuracy'), row=1, col=2)

fig.update_layout(
    title="Curvas de Pérdida y Precisión del Modelo",
    xaxis_title="Épocas",
    yaxis_title="Valor",
    showlegend=True,
    height=500, width=1000
)

fig.show()


### Resultados e Interpretación 2

Las gráficas de pérdida y precisión reflejan claramente los problemas identificados durante el entrenamiento del modelo. En la primera gráfica, correspondiente a la pérdida (Loss), se observa una disminución progresiva tanto en el conjunto de entrenamiento como en el conjunto de validación. Sin embargo, las pérdidas se mantienen en valores relativamente altos, alrededor de 2.1, lo que sugiere que el modelo no logró optimizar correctamente los pesos para mejorar las predicciones. Esto indica que el aprendizaje del modelo es insuficiente y que no está capturando de manera efectiva los patrones presentes en los datos.

En la segunda gráfica, que muestra la precisión (Accuracy), se aprecia un crecimiento muy lento en ambas curvas. La precisión en el conjunto de entrenamiento apenas llega a 16.5% después de 10 épocas, mientras que la precisión en el conjunto de validación muestra ligeras fluctuaciones, alcanzando un máximo de 25.4%. Estas cifras indican que el modelo no está aprendiendo adecuadamente y que la arquitectura actual no es capaz de generalizar bien en los datos de validación. Además, la oscilación en la precisión del conjunto de validación refleja que el modelo está teniendo dificultades para estabilizar el aprendizaje, posiblemente debido a una configuración subóptima de los hiperparámetros.


### Conclusión


En este proyecto, se llevó a cabo el desarrollo de un modelo de Red Neuronal Convolucional (CNN) para la clasificación de imágenes utilizando el dataset CIFAR-10. A lo largo del proceso, se realizaron diversos experimentos y ajustes, incluyendo la implementación de Data Augmentation, la sintonización de hiperparámetros con keras_tuner, y el uso de técnicas avanzadas como transfer learning con ResNet50. Sin embargo, los resultados obtenidos evidencian que la configuración inicial del modelo no fue óptima, lo que afectó significativamente su rendimiento, como se refleja en las bajas precisiones tanto en el conjunto de entrenamiento como en el de validación, junto con pérdidas elevadas que indican dificultades en la optimización del aprendizaje.

El análisis de las métricas y las curvas de pérdida y precisión mostró que, a pesar de los esfuerzos por mejorar el modelo, los ajustes realizados no fueron suficientes para capturar de manera eficiente los patrones presentes en los datos. Además, el proceso de entrenamiento resultó ser computacionalmente costoso, con tiempos excesivos por época y sin una mejora significativa en el desempeño. Esto sugiere que la arquitectura utilizada y los hiperparámetros seleccionados no fueron adecuados para resolver el problema de clasificación de manera efectiva.

A pesar de los desafíos enfrentados, este proyecto ofrece valiosos aprendizajes sobre la importancia de ajustar correctamente las configuraciones del modelo y seleccionar las arquitecturas apropiadas para cada tarea. La experiencia adquirida destaca la necesidad de iterar sobre las configuraciones del modelo, probar diferentes enfoques y ajustar los hiperparámetros de manera más precisa. En futuros trabajos, se recomienda explorar arquitecturas más profundas, realizar una validación más exhaustiva de los hiperparámetros, y utilizar modelos preentrenados para acelerar el aprendizaje y mejorar la precisión. 
